In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Extracting Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Extracting Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Extracting Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Extracting Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Extracting Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [6]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent': '<Enter User Agent here>', 'Accept-Language': 'en-US, en;q=0.5'})


    # The webpage URL
    URL = "https://www.amazon.com/s?k=laptop&crid=I0YJPQWBMW7B&sprefix=laptop%2Caps%2C424&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [7]:
amazon_df

,title,price,rating,reviews,availability
0,KOORUI Portable Monitor 15.6 Inch 1080P FHD Po...,,4.3 out of 5 stars,"1,497 ratings",In Stock
1,LG Gram (2022) 16Z90Q Ultra Lightweight Laptop...,,4.7 out of 5 stars,4 ratings,Only 3 left in stock (more on the way)
2,"Lenovo 2022 Newest Ideapad 3 Laptop, 15.6"" HD ...",,4.4 out of 5 stars,"1,933 ratings",In Stock
3,"Acer Aspire 5 A515-46-R3UB | 15.6"" Full HD IPS...",,4.5 out of 5 stars,"39,480 ratings",In Stock
4,"Acer Aspire 5 A515-56-32DK Slim Laptop - 15.6""...",,4.3 out of 5 stars,"4,124 ratings",In Stock
5,"Acer Aspire 5 A515-46-R3UB | 15.6"" Full HD IPS...",,4.5 out of 5 stars,"39,480 ratings",In Stock
6,"2020 HP 14"" HD (1366 x 768) Thin and Light Lap...",,4.4 out of 5 stars,756 ratings,Only 3 left in stock - order soon
7,"MSI Katana GF66 15.6"" 144Hz FHD Gaming Laptop:...",,4.5 out of 5 stars,"1,584 ratings",In Stock
8,"SAMSUNG 14” Galaxy Book3 Pro Laptop Computer, ...",,Previous page of related Sponsored Products,,Only 3 left in stock (more on the way)
9,"HP 2023 15'' HD IPS Laptop, Windows 11, Intel ...",,4.3 out of 5 stars,9 ratings,In Stock.
